In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import dask.dataframe as dd
from dask.multiprocessing import get
import cv2
from tqdm import tqdm

import matplotlib.pyplot as plt

tqdm.pandas()

In [2]:
label_path = "/run/media/viper/LSP/Dataset/Affect/labels.csv"
df = pd.read_csv(label_path, sep=",")
df.head()

,pth,label
0,anger/image0000006.jpg,surprise
1,anger/image0000007.jpg,anger
2,anger/image0000012.jpg,anger
3,anger/image0000035.jpg,fear
4,anger/image0000060.jpg,anger


In [3]:
def get_hw(row):
    img = cv2.imread("/run/media/viper/LSP/Dataset/Affect/" + row)
    return f"{img.shape[0]},{img.shape[1]},{img.shape[2]}"

In [4]:
df["hw"] = df["pth"].progress_apply(get_hw)

100%|██████████| 31002/31002 [12:01<00:00, 42.94it/s]


In [5]:
counter = 0
for i in range(len(df)):
    v = df.iloc[i]["hw"].split(",")
    if v[0] != v[1]:
        counter += 1
print(counter)

6


In [44]:
chunksize = 10
se_index = np.linspace(0, len(df), chunksize).astype(int)
for i in range(chunksize - 1):
    X_affectnet = np.zeros((se_index[i+1]-se_index[i], 224, 224, 3))
    for j in tqdm(range(se_index[i], se_index[i+1])):
        img = cv2.imread("/run/media/viper/LSP/Dataset/Affect/" + df.iloc[i]["pth"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        X_affectnet[j-se_index[i],:,:,:] = resized
    np.save("/run/media/viper/LSP/Dataset/IRP_data/Affect/X_affectnet"+ str(i) + ".npy", X_affectnet, allow_pickle=True)

100%|██████████| 3445/3445 [04:13<00:00, 13.62it/s]


In [7]:
Y = pd.get_dummies(df["label"])

In [10]:
Y["pth"] = 0

In [13]:
chunksize = 10
se_index = np.linspace(0, len(df), chunksize).astype(int)
for i in range(chunksize-1):
    for j in tqdm(range(se_index[i], se_index[i+1])):
        Y.iloc[j, -1] = "X_affectnet" + str(i) + ".npy"

  0%|          | 0/3444 [00:00<?, ?it/s]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [7]:
chunksize = 10
se_index = np.linspace(0, len(df), chunksize).astype(int)
for i in range(chunksize - 1):
    # Y_affectnet = np.zeros((se_index[i+1]-se_index[i], 8))
    Y_affectnet = Y_np[se_index[i]:se_index[i+1],:]
    np.save("/run/media/viper/LSP/Dataset/IRP_data/Affect/Y_affectnet"+ str(i) + ".npy", Y_affectnet, allow_pickle=True)